In [1]:
import numpy as np
import pandas as pd
import statsrat as sr
import edit_distance
from statsrat import rw
from statsrat.expr.predef.cat import fast

In [9]:
# Indicate whether the learning model should be fit.
fit_model = True

# Define learning model.
model = rw.model(name = 'CompAct_eta0',
                 fbase = rw.fbase.elem,
                 fweight = rw.fweight.from_aux_norm,
                 lrate = rw.lrate.from_aux_norm,
                 aux = rw.aux.gradcomp_eta0)

# Define time limit for model fitting (in old R code was Inf, but is 10 by default in current code).
max_time = 120

In [10]:
(ds_spring2020, summary_spring2020) = fast.read_csv(path = 'fast_data_spring/learning',
                                                    x_col = ['cue1', 'cue2'],
                                                    resp_col = ['trial_resp.keys', 'test_resp.keys'],
                                                    resp_map = {'h' : 'cati', 'g' : 'catii', 'c' : 'cat1', 'm' : 'cat2', 's' : 'cat3', 'r' : 'cat4'},
                                                    ident_col = 'sonaID',
                                                    conf_col = 'conf_rating.response',
                                                    n_final = 8)


print(summary_spring2020)

wrong number of trials for file fast_data_spring/learning/20_spring_faces_2020_Jan_28_1526.csv
trials found: 194
trials expected: 204
Usecols do not match columns, columns expected but not found: ['cue1', 'trial_resp.keys', 'cue2', 'test_resp.keys', 'conf_rating.response']
Usecols do not match columns, columns expected but not found: ['test_resp.keys', 'conf_rating.response']
Usecols do not match columns, columns expected but not found: ['cue1', 'trial_resp.keys', 'cue2', 'test_resp.keys', 'conf_rating.response']
Usecols do not match columns, columns expected but not found: ['test_resp.keys', 'conf_rating.response']
Usecols do not match columns, columns expected but not found: ['cue1', 'trial_resp.keys', 'cue2', 'test_resp.keys', 'conf_rating.response']
Usecols do not match columns, columns expected but not found: ['cue1', 'trial_resp.keys', 'cue2', 'test_resp.keys', 'conf_rating.response']
Usecols do not match columns, columns expected but not found: ['cue1', 'trial_resp.keys', 'cue2'

In [17]:
# **** import survey data ****
df_surveys = pd.read_csv('fast_data_spring/survey/JonesSec9.csv')
df_surveys.rename(columns = {'IDCode': 'ident',
                   'mjon1': 'anx1', # In the past 7 days...I felt fearful
                   'mjon2': 'anx2', # In the past 7 days...I found it hard to focus on anything other than my anxiety
                   'mjon3': 'anx3', # In the past 7 days...My worries overwhelmed me
                   'mjon4': 'anx4', # In the past 7 days...I felt uneasy
                   'mjon5': 'anx5', # In the past 7 days...I felt nervous
                   'mjon6': 'anx6', # In the past 7 days...I felt like I needed help for my anxiety
                   'mjon7': 'anx7', # In the past 7 days...I felt anxious
                   'mjon8': 'anx8', # In the past 7 days...I felt tense
                   'mjon9': 'ang1', # In the past 7 days...I was irritated more than people knew
                   'mjon10': 'ang2', # In the past 7 days...I felt angry
                   'mjon11': 'ang3', # In the past 7 days...I felt like I was ready to explode
                   'mjon12': 'ang4', # In the past 7 days...I was grouchy
                   'mjon13': 'ang5', # In the past 7 days...I felt annoyed
                   'mjon14': 'dep1', # In the past 7 days...I felt worthless
                   'mjon15': 'dep2', # In the past 7 days...I felt that I had nothing to look forward to
                   'mjon16': 'dep3', # In the past 7 days...I felt helpless
                   'mjon17': 'dep4', # In the past 7 days...I felt sad
                   'mjon18': 'dep5', # In the past 7 days...I felt like a failure
                   'mjon19': 'dep6', # In the past 7 days...I felt depressed
                   'mjon20': 'dep7', # In the past 7 days...I felt unhappy
                   'mjon21': 'dep8', # In the past 7 days...I felt hopeless
                   'mjon22': 'pos1', # In the past 7 days...I felt cheerful
                   'mjon23': 'pos2', # In the past 7 days...I felt attentive
                   'mjon24': 'pos3', # In the past 7 days...I felt delighted
                   'mjon25': 'pos4', # In the past 7 days...I felt happy
                   'mjon26': 'pos5', # In the past 7 days...I felt joyful
                   'mjon27': 'pos6', # In the past 7 days...I felt enthusiastic
                   'mjon28': 'pos7', # In the past 7 days...I felt determined
                   'mjon29': 'pos8', # In the past 7 days...I felt interested
                   'mjon30': 'pos9', # In the past 7 days...I was thinking creatively
                   'mjon31': 'pos10', # In the past 7 days...I liked myself
                   'mjon32': 'pos11', # In the past 7 days...I felt peaceful
                   'mjon33': 'pos12', # In the past 7 days...I felt good-natured
                   'mjon34': 'pos13', # In the past 7 days...I felt useful
                   'mjon35': 'pos14', # In the past 7 days...I felt understood
                   'mjon36': 'pos15'}, # In the past 7 days...I felt content
                  inplace = True)
df_surveys['ident'] = df_surveys['ident'].str.lower()
print(df_surveys.head())

       ident  Section Sum  anx1  anx2  anx3  anx4  anx5  anx6  anx7  anx8  \
0  1211hamaj         85.0   2.0   3.0   2.0   2.0   3.0   3.0   3.0   3.0   
1  0106raber        111.0   2.0   4.0   4.0   3.0   4.0   3.0   5.0   4.0   
2  0704jafai        104.0   3.0   3.0   3.0   3.0   4.0   4.0   3.0   2.0   
3  0424pahig        117.0   4.0   2.0   3.0   3.0   3.0   3.0   4.0   3.0   
4  0905gimac         91.0   2.0   2.0   3.0   3.0   3.0   2.0   3.0   2.0   

   ...  pos6  pos7  pos8  pos9  pos10  pos11  pos12  pos13  pos14  pos15  
0  ...   3.0   2.0   2.0   2.0    2.0    3.0    3.0    3.0    2.0    2.0  
1  ...   3.0   3.0   3.0   4.0    3.0    3.0    3.0    2.0    3.0    3.0  
2  ...   4.0   4.0   4.0   2.0    4.0    3.0    4.0    3.0    3.0    4.0  
3  ...   5.0   4.0   4.0   4.0    4.0    4.0    4.0    4.0    3.0    4.0  
4  ...   3.0   3.0   3.0   2.0    3.0    3.0    4.0    4.0    3.0    3.0  

[5 rows x 38 columns]
0
1
2
3
miau
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
mia

In [ ]:
# **** drop people not found in FAST data, and fix IDs ****

# https://pypi.org/project/edit-distance/
fast_ident = summary_spring2020.index.values # list of IDs found in the FAST data
for i in df_surveys.index:
    ident_i = df_surveys['ident'].loc[i]
    if ident_i in fast_ident:
        print('miau')
    else:
        df_surveys.drop(index = i, inplace = True)
print(df_surveys.shape[0])

In [6]:
ref = [1, 'a2', 3, 4]
hyp = [1, 2, 4, 5, 6]
sm = edit_distance.SequenceMatcher(a=ref, b=hyp)
sm.get_opcodes()
sm.ratio()
sm.get_matching_blocks()
print(sm.distance())

4


In [ ]:
print(summary_spring2020.shape[0])

print(summary_spring2020['rel_irl'].min())
print(summary_spring2020['rel_irl'].max())
print(summary_spring2020['rel_irl'].mean())

print(summary_spring2020['threat_benign_os'].min())
print(summary_spring2020['threat_benign_os'].max())
print(summary_spring2020['threat_benign_os'].mean())